<a href="https://colab.research.google.com/github/rjz46/Cornell-Conversational-Analysis-Toolkit/blob/master/datasets/reddit_coarse_discourse-corpus/A1E_Upvote_Toxicity_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and setup

In [0]:
reddit_corpus = convokit.Corpus(filename="corpus")
from tox_dictionary import toxicity_scores as tox

In [0]:
from IPython.display import display as disp
from zipfile import ZipFile
import json
import matplotlib.pyplot as plt
import pandas as pd
import os
import collections

In [0]:
#@title Change working dir if necessary before importing convokit

# UNCOMMENT AND RUN ONCE PER KERNEL, listdir() output should include "convokit"
# os.chdir('../..')

print(os.listdir())

In [0]:
import convokit

In [0]:
reddit_corpus = convokit.Corpus(filename="datasets/reddit_coarse_discourse-corpus/corpus")

# Sequences

In [0]:
subreddit_counter = collections.Counter()
convos = reddit_corpus.conversations


for convoutt in convos: 
    #print(convos[convoutt])
    subrd = convos[convoutt].meta["subreddit"]
    subreddit_counter.update({subrd:1})
top_25_subreddits = dict(subreddit_counter.most_common(25))

print(top_25_subreddits.keys())

dict_keys(['askreddit', 'leagueoflegends', 'buildapc', 'explainlikeimfive', 'trees', 'pcmasterrace', 'gaming', 'techsupport', 'electronic_cigarette', 'relationships', 'tipofmytongue', 'hearthstone', 'atheism', 'jailbreak', 'summonerschool', 'personalfinance', 'reddit.com', 'whowouldwin', 'destinythegame', 'movies', 'pokemon', 'starcraft', 'malefashionadvice', 'askscience', 'loseit'])


In [0]:
top_level_posts = {}
counter = 0
in_top_25 = {}

for x in convos:
    if convos[x].meta["subreddit"] in top_25_subreddits.keys():
        in_top_25[x] = convos[x]

In [0]:
utts = reddit_corpus.utterances

print(len(in_top_25))

2021


# Upvotes

In [0]:
utt_ups_tox = {}


for x in convos:
    utt_ups_tox[x] = [utts[x].meta['ups'], tox[x]]


In [0]:
print(utt_ups_tox)

{'t3_1bx6qw': [3, 0.21341012], 't3_omv7p': [6, 0.6519473], 't3_259tbh': [26, 0.22255301], 't3_16h61h': [6, 0.1913129], 't3_35igzp': [3, 0.10680472], 't3_195abs': [3, 0.0], 't3_1uz5j7': [1, 0.2829675], 't3_20pv9h': [6, 0.11846728], 't3_25hor7': [None, 0.34233686], 't3_27aan0': [3, 0.090869814], 't3_289m0u': [1, 0.015565875], 't3_2fu170': [5, 0.20046036], 't3_2hdp7c': [None, 0.057118468], 't3_2kmipp': [14, 0.0], 't3_2rww3m': [4, 0.14246689], 't3_2x3amn': [1, 0.17399204], 't3_300uc4': [None, 0.065979816], 't3_335w7l': [4, 0.3749667], 't3_33huo1': [None, 0.0], 't3_3422rx': [1, 0.17642066], 't3_36easv': [1, 0.066829436], 't3_3bxzpg': [2, 0.21681778], 't3_3cle3y': [None, 0.0], 't3_3coc19': [3, 0.08348231], 't3_3j07o7': [84, 0.0], 't3_3wwrop': [3, 0.21229206], 't3_3zrvv2': [2, 0.20272402], 't3_4024ia': [1, 0.071258835], 't3_44s5t4': [3, 0.42424884], 't3_4b3wya': [3, 0.029024871], 't3_4fa79x': [2, 0.27546665], 't3_4hrd19': [1, 0.0], 't3_4kp6gm': [11, 0.0], 't3_1bwfs2': [2, 0.07020012], 't3_11m